In [1]:
import os
import numpy as np
import torch
import torch.nn.functional as F

from detection_utils import random_label
from detection_evaluate import single_metric_fpr_tpr, combined_metric_fpr_tpr
from detection_baseline import targeted_vals, untargeted_vals

# from utils import transform
import torch.optim as optim

import torchvision
from torchvision import transforms
import torch.utils.data as data

from tqdm.notebook import tqdm
from timm.data.constants import IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD
import json

In [2]:
# Path to the folder where the pretrained models are saved
CHECKPOINT_PATH = "/Users/albertwen/Downloads/mae_data/resnet50.pth"

# Setting the seed
# pl.seed_everything(42)

# Ensure that all operations are deterministic on GPU (if used) for reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Fetching the device that will be used throughout this notebook
device = torch.device("cpu") if not torch.cuda.is_available() else torch.device("cuda:0")
print("Using device", device)

pretrained_model = torchvision.models.resnet50(pretrained=True)
pretrained_model = pretrained_model.to(device)

if CHECKPOINT_PATH:
    if os.path.isfile(CHECKPOINT_PATH):
        checkpoint = torch.load(CHECKPOINT_PATH) 
        # model.load_state_dict(checkpoint['state_dict'])  #['state_dict']
        pretrained_model.load_state_dict(checkpoint)  #['state_dict']
        print("=> load checkpoint found at {}".format(CHECKPOINT_PATH))
        # print("=> loaded checkpoint '{}' (epoch {})"
        #       .format(args.resume, checkpoint['epoch']))
    else:
        print("=> no checkpoint found at '{}'".format(CHECKPOINT_PATH))
# No gradients needed for the network
pretrained_model.eval()
for p in pretrained_model.parameters():
    p.requires_grad = False

Using device cpu
=> load checkpoint found at /Users/albertwen/Downloads/mae_data/resnet50.pth


In [5]:
fpr = 0.1 # can be [0.1, 0.2]
untargeted_step_threshold = 1000
criterions = {0.1: (0.190, 35, untargeted_step_threshold), 0.2: (1.77, 22, untargeted_step_threshold)} # pre-defined in paper's code
dataset = "imagenet"
attacks = ["fgsm_4", "fgsm_8", "pgd_8"]
real_d = "/Users/albertwen/Downloads/mae_data/ImageNet-Data/val"
adv_d = "/Users/albertwen/Downloads/mae_data/ImageNet-Data/attack/val"

noise_radius = 0.1
targeted_lr = 0.005 

# TODO: go through paper and re-evaluate , different result for pgd and cw, better for cw to try for that
targeted_radius = 0.03
untargeted_radius = 0.03
untargeted_lr = 0.1

det_opt = "targeted" # can be ["targeted", "untargeted"]

In [4]:
# testing for real vs attacked images
real_steps = targeted_vals(pretrained_model, "imagenet", "real", real_d, adv_d, targeted_lr=targeted_lr, t_radius=targeted_radius)
# real_steps

KeyboardInterrupt: 

In [9]:
len(real_steps) # there are 101 total images in this dataset

91

In [16]:
real_steps_shark = targeted_vals(pretrained_model, "imagenet", "real", "/Users/albertwen/Downloads/mae_data/ImageNet-Data/test_val3/", adv_d, targeted_lr=targeted_lr, t_radius=targeted_radius)
real_steps_shark

# TODO: test the prediction power on this

array([], dtype=float64)

In [15]:
real_steps_shark = targeted_vals(pretrained_model, "imagenet", "real", "/Users/albertwen/Downloads/mae_data/ImageNet-Data/test_val2", adv_d, targeted_lr=targeted_lr, t_radius=targeted_radius)
real_steps_shark

array([], dtype=float64)

In [10]:
fgsm4_steps = targeted_vals(pretrained_model, "imagenet", "fgsm_4", real_d, real_d, targeted_lr=targeted_lr, t_radius=targeted_radius)
fgsm4_steps

# fake testing, passed in real dir to see if fgsm4_steps = real_steps, .e. attack pipeline is same

this is the number of successes in targeted detection -91


array([0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [12]:
correct_fgsm4_steps = targeted_vals(pretrained_model, "imagenet", "fgsm_4", real_d, adv_d, targeted_lr=targeted_lr, t_radius=targeted_radius)
correct_fgsm4_steps

# fake testing, passed in real dir to see if fgsm4_steps = real_steps, .e. attack pipeline is same

this is the number of successes in targeted detection -44


array([0., 0., 0., 0., 0., 0.])

In [5]:
# targeted single metric
single_metric_fpr_tpr(fpr, 
                      criterions, 
                      pretrained_model, 
                      dataset, 
                      attacks, 
                      real_d, 
                      adv_d, 
                      noise_radius, 
                      targeted_lr, 
                      targeted_radius, 
                      untargeted_lr, 
                      untargeted_radius, 
                      opt="targeted")

KeyboardInterrupt: 

In [4]:
single_metric_fpr_tpr(fpr, 
                      criterions, 
                      pretrained_model, 
                      dataset, 
                      attacks, 
                      real_d, 
                      adv_d, 
                      noise_radius, 
                      targeted_lr, 
                      targeted_radius, 
                      untargeted_lr, 
                      untargeted_radius, 
                      opt="untargeted")

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/albertwen/opt/anaconda3/envs/ml/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3433, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/y9/l2z6fxk97j32t00jbm20rm6w0000gn/T/ipykernel_41765/25867023.py", line 1, in <module>
    single_metric_fpr_tpr(fpr,
  File "/Users/albertwen/GitHub/MAE-Reverse-Adversarial/detection_evaluate.py", line 32, in single_metric_fpr_tpr
    target = untargeted_vals(model, dataset, "real", real_dir, adv_dir, untargeted_lr, u_radius)
  File "/Users/albertwen/GitHub/MAE-Reverse-Adversarial/detection_baseline.py", line 232, in untargeted_vals
  File "/Users/albertwen/GitHub/MAE-Reverse-Adversarial/detection_baseline.py", line 104, in untargeted_detection
    if attack == "real":
  File "/Users/albertwen/opt/anaconda3/envs/ml/lib/python3.8/site-packages/torch/nn/modules/module.py", line 889, in _call_impl
    result = self.forward(*input, **kwargs)
  File "/